In [4]:
from tasks.linkage import Linkage

In [18]:
import gomea
import numpy as np

# Custom fitness function resembling the concatenated deceptive trap function of size k
class CustomTrapFunction(gomea.fitness.BBOFitnessFunctionDiscrete):
    # Any members must be assigned in __new__ to make them accessible during instantiation of superclass
    def __new__(self, number_of_variables, k, value_to_reach):
        assert( number_of_variables % k == 0 )
        self.k = k
        return super().__new__(self,number_of_variables,value_to_reach)

    def objective_function(self, objective_index, variables):
        f = 0
        for i in range(0,self.number_of_variables,self.k):
            trap_variables = variables[i:i+self.k]
            unitation = np.sum(trap_variables)
            if unitation == self.k:
                f += unitation
            else:
                f += self.k - unitation - 1
        return f

dim = 10
lm = gomea.linkage.BlockMarginalProduct(block_size=5)
fd = CustomTrapFunction(dim,k=5,value_to_reach=dim)
dgom = gomea.DiscreteGOMEA(fitness=fd,linkage_model=lm,max_number_of_evaluations=100000)
result = dgom.run()
result.printAllStatistics()
result.printFinalStatistics()

All statistics:
best_cons_val : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
population_size : [2, 4, 4, 4, 8, 8]
population_index : [0, 1, 1, 1, 2, 2]
eval_time : [0.00013299999999999998, 0.00025899999999999995, 0.0003279999999999999, 0.00038799999999999984, 0.0005849999999999995, 0.0006479999999999994]
best_obj_val : [8.0, 8.0, 8.0, 8.0, 8.0, 10.0]
time : [0.000165, 0.000303, 0.00037799999999999997, 0.000443, 0.000685, 0.00078]
evaluations : [6.0, 20.0, 28.0, 35.0, 59.0, 65.0]
generation : [1, 1, 2, 3, 1, 1]
Final statistics:
best_cons_val : 0.0
population_size : 8
population_index : 2
eval_time : 0.0006479999999999994
best_obj_val : 10.0
time : 0.00078
evaluations : 65.0
generation : 1


In [3]:
import random
random.choice(range(0))


IndexError: range object index out of range

In [6]:
import gomea
import numpy as np
# Custom fitness function resembling the concatenated deceptive trap function of size k
class CustomTrapFunction(gomea.fitness.BBOFitnessFunctionDiscrete):
    # Any members must be assigned in __new__ to make them accessible during instantiation of superclass
    def __new__(self, number_of_variables, k,l):
        self.k = k
        self.l = l
        return super().__new__(self,number_of_variables)
        
    def objective_function(self, objective_index, variables):
        f = 0
        s = ''
        for i in range(0,self.number_of_variables):
            s += str(variables[i])
        out = self.l.step(s)
        f = out[1]
        #print(f)
        return f

dim = 8
l = Linkage(dim,0.2)
l.seed(2)
l.reset()
lm = gomea.linkage.LinkageTree()
fd = CustomTrapFunction(dim,k=1,l=l)
dgom = gomea.DiscreteGOMEA(fitness=fd,linkage_model=lm,max_number_of_evaluations=10000)
result = dgom.run()
result.printAllStatistics()
result.printFinalStatistics()

ERROR! Session/line number was not unique in database. History logging moved to new session 82


In [2]:
import numpy as np

def fitness(bitstring):
    """ Calculate the fitness of a bitstring. Fitness is the sum of XOR of pairs of bits. """
    return np.sum([bitstring[i] ^ bitstring[i+1] for i in range(0, len(bitstring), 2)])

def special_crossover(parent1, parent2):
    """ Perform special crossover where pairs of bits are treated as a unit. """
    offspring = np.empty_like(parent1)
    for i in range(0, len(parent1), 2):
        if np.random.rand() < 0.5:
            offspring[i:i+2] = parent1[i:i+2]
        else:
            offspring[i:i+2] = parent2[i:i+2]
    return offspring

def uniform_crossover(parent1, parent2):
    """ Perform uniform crossover. """
    offspring = np.array([parent1[i] if np.random.rand() < 0.5 else parent2[i] for i in range(len(parent1))])
    return offspring

def mutate(bitstring, mutation_rate=1):
    """ Mutate a bitstring with a given mutation rate. """
    for i in range(len(bitstring)):
        if np.random.rand() < mutation_rate / len(bitstring):
            bitstring[i] = 1 - bitstring[i]
    return bitstring

In [3]:
import matplotlib.pyplot as plt

def genetic_algorithm_with_mutation_rate(n, crossover_function, mutation_rate, num_runs=500):
    """ Run a (5+1) genetic algorithm with the given crossover function and mutation rate. """
    total_generations = 0
    for _ in range(num_runs):
        population = [np.random.randint(0, 2, 2*n) for _ in range(5)]
        generations = 0
        while True:
            generations += 1
            parents_indices = np.random.choice(len(population), 2, replace=False)
            parents = [population[parents_indices[0]], population[parents_indices[1]]]

            # Apply crossover if a crossover function is provided, else skip to mutation
            if crossover_function:
                offspring = crossover_function(parents[0], parents[1])
            else:
                offspring = np.copy(parents[0])  # choose a random parent as offspring for mutation only
            
            offspring = mutate(offspring, mutation_rate=mutation_rate)
            offspring_fitness = fitness(offspring)
            min_fitness_individual_index = np.argmin([fitness(individual) for individual in population])
            if offspring_fitness > fitness(population[min_fitness_individual_index]):
                population[min_fitness_individual_index] = offspring
            if max(map(fitness, population)) == n:
                break
        total_generations += generations
    return total_generations / num_runs

# Mutation rates
n = 100
bitstring_length = 2 * n  # Size of the bitstring
mutation_rates = np.linspace(0.5, 1.5, 21)

# Run simulations and collect data
avg_generations_special = [genetic_algorithm_with_mutation_rate(n, special_crossover, mr) for mr in mutation_rates]
avg_generations_uniform = [genetic_algorithm_with_mutation_rate(n, uniform_crossover, mr) for mr in mutation_rates]
avg_generations_mutation_only = [genetic_algorithm_with_mutation_rate(n, None, mr) for mr in mutation_rates]

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(mutation_rates, avg_generations_special, label='Special Crossover')
plt.plot(mutation_rates, avg_generations_uniform, label='Uniform Crossover')
plt.plot(mutation_rates, avg_generations_mutation_only, label='Mutation Only')
plt.xlabel('Mutation Rate')
plt.ylabel('Average Generations')
plt.title('Average Generations vs Mutation Rate')
plt.legend()
plt.show()

In [3]:
for i in range(10,-1,-1):
    print(i)

10
9
8
7
6
5
4
3
2
1
0


In [2]:
import numpy as np

# Example NumPy array of 0s and 1s
array = np.array([0, 1, 1, 0, 1, 0])

# Convert the NumPy array to a string of 0s and 1s
array_str = ''.join(str(x) for x in array)
print(array_str)

011010


In [27]:

import numpy as np
from scipy.special import comb
from itertools import combinations

def calculate_block_fitness(block):
    k = len(block)
    block_fitness = 0

    # Loop through levels
    for level in range(1, k + 1):
        level_contributions = []
        # Generate all combinations for the current level
        for bits in combinations(range(k), level):
            # Generate a random contribution for each combination
            contribution = np.random.rand()
            level_contributions.append(contribution)
        
        # Standardize the contributions for this level to sum to 1
        level_contributions = np.array(level_contributions) / sum(level_contributions)
        
        # Calculate the contribution to the block fitness from this level
        for i, bits in enumerate(combinations(range(k), level)):
            if all(block[b] for b in bits):  # Check if all bits in this combination are 1
                block_fitness += level_contributions[i]

    return block_fitness

def total_fitness(bitstring, k, n):
    assert len(bitstring) == k * n, "Bitstring size must be k * n"
    total_fitness = 0
    
    # Split the bitstring into n blocks and calculate fitness for each block
    for i in range(n):
        block = bitstring[i*k : (i+1)*k]
        total_fitness += calculate_block_fitness(block)
    
    return total_fitness

# Example usage
k, n = 3, 2  # 3 bits per block, 2 blocks
bitstring = np.random.randint(2, size=k*n)
fitness_value = total_fitness(bitstring, k, n)
bitstring, fitness_value

(array([0, 1, 1, 0, 0, 1]), 1.4846876382934808)